# Kubernetesリソースの作成

## GitHubからYAMLを取得する
プロジェクトのGitHubからGKEでWordPressを動かすためのYAMLファイルを取得する。

In [2]:
branch_name="master"
!rm -rf k8s-wordpress-mysql
!git clone -b {branch_name} https://github.com/topselc4ri/k8s-wordpress-mysql.git

Cloning into 'k8s-wordpress-mysql'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 71 (delta 31), reused 42 (delta 14), pack-reused 0
Unpacking objects: 100% (71/71), done.


## [オプション] kubectlの設定
GKEクラスタの構築から続けて実行する場合は、この手順を省略可能。

In [6]:
cluster_name = "topse-cluster"
region="us-west1"
!gcloud container clusters get-credentials {cluster_name} --region {region}
!curl -LO https://storage.googleapis.com/kubernetes-release/release/$(curl -s https://storage.googleapis.com/kubernetes-release/release/stable.txt)/bin/linux/amd64/kubectl
!chmod +x ./kubectl
!sudo mv ./kubectl /usr/local/bin/kubectl
!kubectl version 

Fetching cluster endpoint and auth data.
kubeconfig entry generated for topse-cluster.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 41.4M  100 41.4M    0     0   101M      0 --:--:-- --:--:-- --:--:--  101M
Client Version: version.Info{Major:"1", Minor:"17", GitVersion:"v1.17.1", GitCommit:"d224476cd0730baca2b6e357d144171ed74192d6", GitTreeState:"clean", BuildDate:"2020-01-14T21:04:32Z", GoVersion:"go1.13.5", Compiler:"gc", Platform:"linux/amd64"}
Server Version: version.Info{Major:"1", Minor:"13+", GitVersion:"v1.13.11-gke.14", GitCommit:"56d89863d1033f9668ddd6e1c1aea81cd846ef88", GitTreeState:"clean", BuildDate:"2019-11-07T19:12:22Z", GoVersion:"go1.12.11b4", Compiler:"gc", Platform:"linux/amd64"}


## オブジェクトを作成する
既に存在する場合は更新になります。

In [18]:
!kubectl apply -f k8s-wordpress-mysql/nfs/gcp-filestore.yaml
!kubectl apply -f k8s-wordpress-mysql/nfs/persistent-volume-claim.yaml

persistentvolume/fileserver created
persistentvolumeclaim/fileserver-claim created


In [4]:
!kubectl apply -f k8s-wordpress-mysql/wordpress/wordpress.yaml
!kubectl apply -f k8s-wordpress-mysql/wordpress/wordpress-backendconfig.yaml
!kubectl apply -f k8s-wordpress-mysql/wordpress/wordpress-service.yaml
!kubectl apply -f k8s-wordpress-mysql/wordpress/wordpress-ingress.yaml

deployment.apps/wordpress created
backendconfig.cloud.google.com/wordpress-backendconfig created
service/wordpress created
ingress.extensions/wordpress-ingress created


## オブジェクトを確認する
各オブジェクトが作成されているかを確認する。

In [11]:
!kubectl get PersistentVolume

NAME         CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                      STORAGECLASS   REASON   AGE
fileserver   1G         RWX            Retain           Bound    default/fileserver-claim                           8m50s


In [12]:
!kubectl get PersistentVolumeClaim

NAME               STATUS   VOLUME       CAPACITY   ACCESS MODES   STORAGECLASS   AGE
fileserver-claim   Bound    fileserver   1G         RWX                           9m18s


In [7]:
!kubectl get pods

NAME                        READY   STATUS    RESTARTS   AGE
wordpress-587fb7799-ptbp6   1/1     Running   0          6m36s


In [10]:
!kubectl get backendconfig

NAME                      AGE
wordpress-backendconfig   8m


In [8]:
!kubectl get service

NAME         TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)        AGE
kubernetes   ClusterIP   10.56.0.1     <none>        443/TCP        15m
wordpress    NodePort    10.56.11.32   <none>        80:30810/TCP   6m46s


In [9]:
!kubectl get ingress

NAME                HOSTS   ADDRESS          PORTS   AGE
wordpress-ingress   *       34.107.205.110   80      7m


ingressにパブリックIPアドレス(ADDRESS)が割り当てられるまでには5分程かかります。
更に5分～10分程待つと、WebブラウザからアクセスしてWordPressの画面が見えるようになる。

# Kubernetesリソースの削除
不要になったKubernetesリソースを削除する。

In [13]:
!kubectl delete -f k8s-wordpress-mysql/wordpress/wordpress-ingress.yaml
!kubectl delete -f k8s-wordpress-mysql/wordpress/wordpress-service.yaml
!kubectl delete -f k8s-wordpress-mysql/wordpress/wordpress-backendconfig.yaml
!kubectl delete -f k8s-wordpress-mysql/wordpress/wordpress.yaml

ingress.extensions "wordpress-ingress" deleted
service "wordpress" deleted
backendconfig.cloud.google.com "wordpress-backendconfig" deleted
deployment.apps "wordpress" deleted


In [19]:
!kubectl delete -f k8s-wordpress-mysql/nfs/persistent-volume-claim.yaml
!kubectl delete -f k8s-wordpress-mysql/nfs/gcp-filestore.yaml

persistentvolumeclaim "fileserver-claim" deleted
persistentvolume "fileserver" deleted
